In [188]:
from operator import concat

# create a connection
import psycopg2
conn=psycopg2.connect(database="dwh_db",
                      user="postgres",
                      password="<PASSWORD>",
                      host="localhost",
                      port="5432")


# Table Creation

In [4]:
conn.rollback()

# Open a cursor to perform database operations
cur = conn.cursor()

# University
cur.execute("""create table dim_university(
               uni_key serial primary key,
                uni_id varchar unique,
                name varchar,
                state varchar,
                city varchar,
                street varchar,
                zip varchar
                )
                """)

#department
cur.execute("""create table dim_department(
                dept_key serial primary key,
                dept_code varchar unique,
                uni_key integer not null references dim_university(uni_key)
                )
                """)

#lecturer
cur.execute("""create table dim_lecturer(
               lecturer_key serial primary key,
                lecturer_id varchar unique,
                lecturer_name varchar,
                lecturer_dept integer not null references dim_department(dept_key),
                rank varchar,
                title varchar)
                """)

#Study Plan
cur.execute("""create table dim_study_plan(
                sp_key serial primary key,
                sp_title varchar,
                sp_code integer unique,
                sp_type varchar,
                branch varchar
                )
                """)

# Students
cur.execute("""create table dim_student if not exists(
                student_key serial primary key,
                student_matno varchar unique,
                student_name varchar,
                sp_key integer not null references dim_study_plan(sp_key)
                )
                """)

#date
cur.execute("""create table dim_date(
                date_key date primary key,
                 day integer not null,
                 month integer not null,
                 semester varchar not null,
                 year integer not null)
                """)

#course
cur.execute("""CREATE TABLE dim_course(
            course_key serial primary key,
            course_id varchar unique,
            course_type varchar,
            ects integer,
            lecturer_key integer not null references dim_lecturer(lecturer_key),
            level varchar,
            department_key integer not null references dim_department(dept_key)
            );
            """)


#grades
cur.execute("""create table fact_grades(
                grade_key serial primary key,
                course_key integer not null references dim_course(course_key),
                lecturer_key integer not null references dim_lecturer(lecturer_key),
                date_key date not null references dim_date(date_key),
                grades numeric,
                student_key integer not null references dim_student(student_key)
                )
                """)
# Make the changes to the database persistent
conn.commit()
# Close cursor and communication with the database
cur.close()
conn.close()

In [99]:
cur=conn.cursor()
conn.rollback()
cur.execute("""alter table dim_student
add column matno   VARCHAR UNIQUE""")

conn.commit()
# Close cursor and communication with the database
cur.close()
conn.close()


In [118]:
cur=conn.cursor()
conn.rollback()
cur.execute("""alter table dim_course
add column course_title   VARCHAR UNIQUE""")

conn.commit()
# Close cursor and communication with the database
cur.close()

In [131]:
cur=conn.cursor()
conn.rollback()
cur.execute("""
ALTER TABLE fact_grades
  DROP COLUMN IF EXISTS passed,
  ALTER COLUMN grades TYPE VARCHAR USING grades::VARCHAR,
  ALTER COLUMN grades DROP NOT NULL;
""")

conn.commit()
# Close cursor and communication with the database
cur.close()

In [155]:
with conn.cursor() as cur:
    # Convert all “failed” strings into the text “5”
    cur.execute("""
        update fact_grades
           SET grades = '5'
         WHERE grades = 'failed';
    """)

conn.commit()


In [179]:
with conn.cursor() as cur:
    cur.execute("""
    ALTER TABLE fact_grades
  ALTER COLUMN grades
    TYPE NUMERIC
    USING grades::NUMERIC;     -- cast the existing text to numbers

    """)

conn.commit()

# Data frame creation for inserting to tables

In [18]:
import json
import pandas as pd
import re

**from the aau_metadata.json, we build the pandas dataframes for:**
* university info
* Study plans
* Lecturers

In [67]:
#university and study plans and Lecturer


with open("aau_metadata.json") as f:
    data=json.load(f)

uni_info = {
    "uni_id": data["id"],
    "name": data["name"],
    "state": data["state"],
    "city": data["city"],
    "street": data["street"],
    "zip": data["zip"]
}

#university data frame
df_university=pd.DataFrame([uni_info])


# study plan data frame
df_bachelor_sp=pd.DataFrame(data["bachelor_study_plans"])
df_bachelor_sp["type"]="bachelor"
df_master_sp=pd.DataFrame(data["master_study_plans"])
df_master_sp["type"]="master"
df_sp=pd.concat([df_bachelor_sp,df_master_sp],ignore_index=True)


# print(df_sp)
# lecturers
df_lecturer=pd.DataFrame(data["lecturers"])


    id                   name             branch      type
0  511  Angewandte Informatik  Technical Studies  bachelor
1  522  Wirtschaftsinformatik         Economoics  bachelor
2  922  Wirtschaftsinformatik          Economics    master
3  911  Angewandte Informatik  Technical Studies    master


**Lectureres Data Cleaning to build:**
* rank
* title
* name of lecturers

In [57]:
# lecturer data cleaning

# lookup tables
RANKS = {
  "O.Univ.-Prof.":     "Full Professor",
  "Assoc.-Prof.":      "Associate Professor",
  "Assoc.Prof.":       "Associate Professor",
  "Postdoc-Ass.":      "Postdoc Assistant",
  # we can add to this dic
}

TITLES = {
  "Dipl.-Ing.": "DI",
  "Dr.":        "Dr",
  "DI.":        "DI",
 # we can add to this dic
}

def lecturer_extraction(st):
    tokens = st.split()
    rank = None
    # ranks
    for length in (2, 1):
        candidate = " ".join(tokens[:length])
        if candidate in RANKS:
            rank = RANKS[candidate]
            tokens = tokens[length:]
            break

    # titles
    titles = []
    while tokens and tokens[0] in TITLES:
        titles.append(TITLES[tokens.pop(0)])

    #remaining  are the name
    name = " ".join(tokens)

    return {"name": name, "rank": rank, "titles": titles}


for idx, row in df_lecturer.iterrows():
    s = lecturer_extraction(row["name"])
    # write back only that one row
    df_lecturer.at[idx, "parsed_name"] = s["name"]
    df_lecturer.at[idx, "rank"]        = s["rank"]
    df_lecturer.at[idx, "titles"]      = ", ".join(s["titles"])

print(df_lecturer)

           id                                         name department  \
0    36866626  Assoc.Prof. Dipl.-Ing. Dr. Klaus Schöffmann       ITEC   
1  2077308266              Postdoc-Ass. Dr. Dragi Kimovski       ITEC   
2   834107405    O.Univ.-Prof. Dipl.-Ing. Dr. Johann Eder        ISYS   
3   772243224            Assoc.-Prof. DI. Dr. Julius Köpke       ISYS   

        parsed_name                 rank  titles  
0  Klaus Schöffmann  Associate Professor  DI, Dr  
1    Dragi Kimovski    Postdoc Assistant      Dr  
2       Johann Eder       Full Professor  DI, Dr  
3      Julius Köpke  Associate Professor  DI, Dr  


**from the aau_courses.json we build the below data frames:**
* courses
* departments

In [119]:
# courses and departments df building
with open("aau_corses.json") as f:
    data=json.load(f)

#courses
df_bachelor=pd.DataFrame(data["bachelor"])
df_bachelor["level"]="bachelor"
df_master=pd.DataFrame(data["master"])
df_master["level"]="master"
df_courses=(pd.concat([df_bachelor,df_master],ignore_index=True))

#departments
dept_from_courses=set(df_courses["department"].unique())
dept_from_lecturer=set(df_lecturer["department"].unique())
all_dept=sorted(dept_from_courses.union(dept_from_lecturer))
df_dept=pd.DataFrame({"department":all_dept})


print(df_courses)

        id                                              title type  \
0  620.220  Einführung in die strukturierte und objektbasi...   VO   
1  621.900                                   Web-Technologien   VO   
2  620.050                                        Datenbanken   VO   
3  620.055                               Datenbanktechnologie   UE   
4  623.254  Current Topics in Information Systems Engineering   VC   
5  624.002  Privatissimum für Diplomand/inn/en und Dissert...   PV   
6  623.250                                   Interoperability   VC   
7  623.500                                   Data Engineering   VC   
8  623.502              Distributed Computing Infrastructures   VC   
9  623.252                                Process Engineering   VC   

     lecturer ECTS department     level  
0    36866626    2       ITEC  bachelor  
1   772243224    2       ISYS  bachelor  
2   772243224    2       ISYS  bachelor  
3   772243224    2       ISYS  bachelor  
4   772243224    3 

***from ....result.json files, we build these data frames:***
* students
* grades
* dates (exam times)


In [158]:
# students and time and grades df building

import re

def normalize_time(s):
    """
    If s is just “YYYY” (4 digits), return “YYYY-01-01”;
    otherwise leave it unchanged.
    """
    if re.fullmatch(r"\d{4}", s):
        return f"{s}-02-01"
    return s


dfs=[]
dates=[]
exam_dfs=[]
filenames=["blockchains_bpm","data_engineering","datenbaken","esop","interop","parallel","process_engineering","webtech"]

for name in filenames:
    with open(f"resultlist_{name}.json") as f:
        data=json.load(f)

    #student‐result list
    df=pd.DataFrame(data["results"])

    #students
    df_s = df[["matno", "name", "studyplan"]]
    dfs.append(df_s)

    #date
    dates.append(normalize_time(data["date"]))

    #grades
    df["course_id"]   = data["course"]
    df["lecturer_id"] = data["examinator"]
    df["exam_date"]   = normalize_time(data["date"])
    exam_dfs.append(df)




## in files "resultlist_interop" and "resultlist_webtech" json syntax is broken. A missing colon ("key" "value" instead of "key": "value")
#students
student_df= pd.concat(dfs, ignore_index=True)
student_df.drop_duplicates(subset="matno", inplace=True)

#grades
grade_df= pd.concat(exam_dfs, ignore_index=True)
# replace "failed" with "5" in the same column:
grade_df['grade'] = grade_df['grade'].replace({'failed': '5'})

# convert that column to numbers in‐place (non‐digits become NaN):
grade_df['grade'] = pd.to_numeric(grade_df['grade'], errors='coerce')


#dates
date_df = pd.DataFrame({"exam_date": dates})
date_df["exam_date"] = pd.to_datetime(date_df["exam_date"])
date_df = date_df.drop_duplicates().sort_values("exam_date")

date_dim = pd.DataFrame({
    "date_key": date_df["exam_date"].dt.date,
    "day":      date_df["exam_date"].dt.day,
    "month":    date_df["exam_date"].dt.month,
    "year":     date_df["exam_date"].dt.year,
})
# define your semester logic however you like:
date_dim["semester"] = date_dim["month"].apply(
    lambda m: "S1" if m <= 6 else "S2"
)

print(grade_df)

       matno              name  grade studyplan course_id lecturer_id  \
0    9000078         Sara Groß    2.0       911   623.254   772243224   
1    9000079       Lara Seidel    1.0       922   623.254   772243224   
2    9000082        Sofie Haas    3.0       911   623.254   772243224   
3    9000083  Sophia Schreiber    2.0       922   623.254   772243224   
4    9000086     Nele Dietrich    2.0       911   623.254   772243224   
..       ...               ...    ...       ...       ...         ...   
283  9000092        Amely Horn    5.0       511   621.900   772243224   
284  9000093        Leon Busch    2.0       522   621.900   772243224   
285  9000096       Jonas Voigt    3.0       511   621.900   772243224   
286  9000097        Luca Sauer    4.0       522   621.900   772243224   
287  9000100    Felix Pfeiffer    5.0       511   621.900   772243224   

      exam_date  
0    2022-06-24  
1    2022-06-24  
2    2022-06-24  
3    2022-06-24  
4    2022-06-24  
..          ...

# Dataframes insertion to tables of DB
Load tables in “dependency order", first those without any FKs


In [35]:
# University table inserting
conn.rollback()
with conn.cursor() as cur:
    for _,row in df_university.iterrows():
        cur.execute("insert into dim_university(uni_id,name, state, city,street, zip) values (%s,%s,%s,%s,%s,%s) on conflict (uni_id) do nothing",
                    (row["uni_id"],
                     row["name"],
                     row["state"],
                     row["city"],
                     row["street"],
                     row["zip"])
                    )
conn.commit()

In [44]:
# department
with conn.cursor() as cur:
    for _,row in df_dept.iterrows():
        cur.execute("""insert into dim_department(dept_code,uni_key)
                        values (%s,
                        (select uni_key from dim_university where uni_id=%s)
                                )
                        on conflict (dept_code) do nothing""",
                    (row["dept_code"],df_university["uni_id"].iat[0]))
conn.commit()

In [63]:
# lecturer
conn.rollback()
with conn.cursor() as cur:
    for _,row in df_lecturer.iterrows():
        cur.execute("""insert into dim_lecturer(lecturer_id, lecturer_name, lecturer_dept,rank,title)
         values (%s,
         %s,
         (select dept_key from dim_department where dept_code=%s),
         %s,
         %s)
         on conflict (lecturer_id) do nothing""",
        (
        row["id"],
        row["parsed_name"],
        row["department"],
        row["rank"],
        row["titles"]
        )
                    )
conn.commit()

In [69]:
# study plans insertion
conn.rollback()
with conn.cursor() as cur:
    for _,row in df_sp.iterrows():
        cur.execute("""
        insert into dim_study_plan(sp_title,sp_code,sp_type,branch)
        values (%s,%s,%s,%s)
        on conflict(sp_code) do nothing
        """ ,
        (
            row["name"],
            row["id"],
            row["type"],
            row["branch"]
        )
        )
conn.commit()

In [104]:
# student
conn.rollback()
with conn.cursor() as cur:
    for _,row in student_df.iterrows():
        cur.execute(
            """
            insert into dim_student(student_name,sp_key,matno)
            values (%s,
                    (select sp_key from dim_study_plan where sp_code=%s),
                    %s
                    )""",
            (
            row["name"],
            row["studyplan"],
            row["matno"]
            )
        )
conn.commit()

In [115]:
# date
conn.rollback()
with conn.cursor() as cur:
    for _,row in date_dim.iterrows():
        cur.execute("""
        insert into dim_date(date_key,day,month,semester,year)
        values (%s,%s,%s,%s,%s)
        """,
        (
            row["date_key"],
            row["day"],
            row["month"],
            row["semester"],
            row["year"]
        )
                    )
conn.commit()

In [123]:
#course
conn.rollback()
with conn.cursor() as cur:
    for _,row in df_courses.iterrows():
        cur.execute("""insert into dim_course(course_id,course_type, ects, lecturer_key,level,department_key,course_title)
        values
        (
        %s,
        %s,
        %s,
        (select lecturer_key from dim_lecturer where lecturer_id=%s),
        %s,
        (select dept_key from dim_department where dept_code=%s ),
        %s) on conflict (course_id) do nothing """,
                    (row["id"],
                     row["type"],
                     row["ECTS"],
                     row["lecturer"],
                     row["level"],
                     row["department"],
                     row["title"]))
conn.commit()

In [162]:
#grades
conn.rollback()
with conn.cursor() as cur:
    for _,row in grade_df.iterrows():
        cur.execute("""
        insert into fact_grades(course_key,lecturer_key,date_key,grades,student_key)
        values (
                (select course_key from dim_course where course_id=%s),
                (select lecturer_key from dim_lecturer where lecturer_id=%s),
                (select date_key from dim_date where date_key=%s),
                %s,
                (select student_key from dim_student where matno=%s))
                """,
             (
                 row["course_id"],
                 row["lecturer_id"],
                 row["exam_date"],
                 row["grade"],
                 row["matno"]
             ))
conn.commit()

In [178]:
conn.rollback()
with conn.cursor() as cur:
    cur.execute("""
    UPDATE fact_grades
   SET grades = regexp_replace(
                  grades,
                  '^[\\s"]+|[\\s"]+$',
                  '',
                  'g'
                );
""")


conn.commit()

# DWH querying

In [154]:
#pandas prefers you pass it an SQLAlchemy engine, got warnings
from sqlalchemy import create_engine
engine = create_engine(
    "postgresql+psycopg2://postgres@localhost:5432/dwh_db"
)

Average grade of each student name by lecturer name, including the average grade
of all students and the average grade of each lecturer.

In [184]:
sql="""
select ds.student_name , dl.lecturer_name,
       avg(fg.grades) as avg_grade
from fact_grades as fg
join dim_student ds on fg.student_key=ds.student_key
join dim_lecturer dl on fg.lecturer_key=dl.lecturer_key
WHERE grades != 'NaN'
group by cube (ds.student_name, dl.lecturer_name)


"""

df = pd.read_sql(sql, con=engine)

pivot = df.pivot_table(
  index="student_name",
  columns="lecturer_name",
  values="avg_grade",
  aggfunc="mean",
  margins=True,         # adds the “All” row & column
  margins_name="All"
)
display(pivot)

lecturer_name,Dragi Kimovski,Johann Eder,Julius Köpke,Klaus Schöffmann,All
student_name,,,,,
Amelie Engel,NaN,5.000000,2.000000,NaN,3.500000
Amelie Meier,2.000000,NaN,2.500000,NaN,2.250000
Amely Horn,NaN,NaN,5.000000,3.0000,4.000000
Amely Lehmann,3.000000,NaN,2.000000,NaN,2.500000
Anna Becker,NaN,NaN,3.000000,NaN,3.000000
...,...,...,...,...,...
Sophie Zimmermann,NaN,NaN,3.000000,2.0000,2.500000
Tim Peters,1.000000,NaN,2.500000,NaN,1.750000
Timm Lang,NaN,NaN,3.500000,2.0000,2.750000


Average grade by the StudyPlan type and the department of the lecturer. Also
include average grade per department and average grade per study plan type.


In [181]:
sql="""
select
    dsp.sp_type as study_plan,
   dd.dept_code as lecturer_department,
       avg(fg.grades) as avg_grades
from fact_grades fg
join dim_student ds on ds.student_key=fg.student_key
join dim_study_plan dsp on ds.sp_key=dsp.sp_key
join dim_lecturer dl on dl.lecturer_key=fg.lecturer_key
join dim_department dd on dd.dept_key=dl.lecturer_dept
WHERE grades != 'NaN'
group by cube (dsp.sp_type, dd.dept_code)
"""

df=pd.read_sql(sql,con=engine)
pivot=df.pivot_table(
    index="study_plan",
    columns="lecturer_department",
    values="avg_grades",
    aggfunc="mean",
    margins=True, margins_name="All"

)
display(pivot)

lecturer_department,ISYS,ITEC,All
study_plan,,,
bachelor,3.096154,3.437500,3.266827
master,2.469388,2.933333,2.701361
All,2.782771,3.185417,2.984094


Average grade of students of each lecturer. Now also include the average grades of
each member of the hierarchy: lecturer  Department  University

In [195]:
sql=("""
select
  COALESCE(dl.lecturer_name, 'All Lecturers')  AS lecturer_name,
  COALESCE(dd.dept_code,      'All Departments') AS lecturer_dept,
  COALESCE(du.name,           'All Universities') AS university_name,
  AVG(fg.grades::NUMERIC)          AS avg_grade
from fact_grades fg
join dim_lecturer  dl on fg.lecturer_key = dl.lecturer_key
join dim_department dd on dl.lecturer_dept = dd.dept_key
join dim_university du on dd.uni_key        = du.uni_key
where fg.grades != 'NaN'
group by rollup(

    du.name,
    dd.dept_code,
    dl.lecturer_name

)
""")

pd.read_sql(sql,con=engine)


,lecturer_name,lecturer_dept,university_name,avg_grade
0,All Lecturers,All Departments,All Universities,2.917857
1,Dragi Kimovski,ITEC,Universität Klagenfurt,2.933333
2,Johann Eder,ISYS,Universität Klagenfurt,2.407407
3,Julius Köpke,ISYS,Universität Klagenfurt,2.851429
4,Klaus Schöffmann,ITEC,Universität Klagenfurt,3.437500
5,All Lecturers,ITEC,Universität Klagenfurt,3.243590
6,All Lecturers,ISYS,Universität Klagenfurt,2.792079
7,All Lecturers,All Departments,Universität Klagenfurt,2.917857
